### Showcase GIL releasing benefits of `fastwave`

In [1]:
import os
import datetime
import random
import string
import numpy as np
from scipy.io import wavfile

class AudioGenerator:
    # Duration is in seconds!
    def __init__(
        self, sample_rate=44100, duration=5, channels=2, prefix="random_audio_"
    ):
        self.sample_rate = sample_rate
        self.duration = duration
        self.channels = channels
        self.file_name = self.generate_random_name(prefix)
        self.file_path = os.path.join(os.getcwd(), self.file_name)
        # Run generation at init!
        self.generate_scipy_audio()

    def generate_scipy_audio(self):
        if self.channels not in [1, 2]:
            raise RuntimeError("Unsupported number of channels!")

        noises = [
            np.random.normal(0, 1, int(self.sample_rate * self.duration))
            for _ in range(self.channels)
        ]
        audio_data = np.column_stack(noises) if self.channels == 2 else noises[0]
        audio_data = (audio_data * 32767).astype(np.int16)
        wavfile.write(self.file_path, self.sample_rate, audio_data)

    def delete_generated_file(self):
        if os.path.exists(self.file_path):
            os.remove(self.file_path)
            # print(f"Deleted file: {self.file_path}")

    def generate_random_name(self, prefix):
        current_datetime = datetime.datetime.now()
        random_suffix = "".join(
            random.choices(string.ascii_uppercase + string.digits, k=3)
        )
        return (
            f"{prefix}{current_datetime.strftime('%Y%m%d%H%M%S')}_{random_suffix}.wav"
        )

In [2]:
import time
import threading
from queue import Queue

def computation_thread(number):
    # Simulating some computation time, should be lower/equal to "regular read"
    time.sleep(0.1) 
    return number

In [3]:
import fastwave

def read_and_compute_fastwave(file_path):

    # Create a queue to communicate between threads
    result_queue = Queue()

    # Read data in a separate thread
    def read_thread():
        nonlocal audio_data
        audio_data = fastwave.read(file_path, mode=fastwave.ReadMode.DEFAULT)
        result_queue.put(audio_data)  # Put the result in the queue

    thread = threading.Thread(target=read_thread)
    thread.start()

    # Perform additional computations in the main thread
    additional_result = computation_thread(42)

    # Wait for the thread to finish
    thread.join()
    # Access the data read by the thread from the queue
    audio_data = result_queue.get()

    return audio_data, additional_result

In [4]:
from scipy.io import wavfile

def read_and_compute_scipy(file_path):

    # Create a queue to communicate between threads
    result_queue = Queue()

    # Read data in a separate thread
    def read_thread():
        nonlocal audio_data
        _, sig = wavfile.read(file_path)
        result_queue.put(sig)  # Put the result in the queue

    thread = threading.Thread(target=read_thread)
    thread.start()

    # Perform additional computations in the main thread
    additional_result = computation_thread(21)

    # Wait for the thread to finish
    thread.join()
    # Access the data read by the thread from the queue
    audio_data = result_queue.get()

    return audio_data, additional_result

In [5]:
import torchaudio

def read_and_compute_torchaudio(file_path):

    # Create a queue to communicate between threads
    result_queue = Queue()

    # Read data in a separate thread
    def read_thread():
        nonlocal audio_data
        sig, _ = torchaudio.load(
            file_path, normalize=False, channels_first=False
        )
        result_queue.put(sig)  # Put the result in the queue

    thread = threading.Thread(target=read_thread)
    thread.start()

    # Perform additional computations in the main thread
    additional_result = computation_thread(37)

    # Wait for the thread to finish
    thread.join()
    # Access the data read by the thread from the queue
    audio_data = result_queue.get()

    return audio_data, additional_result

In [10]:
audio_generator = AudioGenerator(sample_rate=44100, duration=60 * 30, channels=2)
print(f"Generated file: {audio_generator.file_path}")
print(f"Duration: {audio_generator.duration} seconds")
print(f"Channels: {audio_generator.channels}")


# Run simple benchmark
def simple_benchmark(function, audio_generator):
    start_time = time.time()

    # Run the benchmark
    audio_data, other = function(audio_generator.file_path)

    end_time = time.time()

    print(f"Running: {function.__name__} ...")
    print(f"Audio: {audio_data.data if isinstance(audio_data, fastwave.AudioData) else audio_data}")
    print(f"Other: {other}")
    print(f"Total Execution Time: {end_time - start_time} seconds")


simple_benchmark(read_and_compute_fastwave, audio_generator)
simple_benchmark(read_and_compute_scipy, audio_generator)
simple_benchmark(read_and_compute_torchaudio, audio_generator)

audio_generator.delete_generated_file()

Generated file: /Users/jiwaszkiewicz/CodeProjects/fastwave/benchmark/random_audio_20240102052119_MP6.wav
Duration: 1800 seconds
Channels: 2
Running: read_and_compute_fastwave ...
Audio: [[ -5103  23502]
 [  4449 -11882]
 [ 26703  20063]
 ...
 [  7883 -29269]
 [ 12201 -30454]
 [ 30524  31554]]
Other: 42
Total Execution Time: 0.1002500057220459 seconds
Running: read_and_compute_scipy ...
Audio: [[ -5103  23502]
 [  4449 -11882]
 [ 26703  20063]
 ...
 [  7883 -29269]
 [ 12201 -30454]
 [ 30524  31554]]
Other: 21
Total Execution Time: 0.10190987586975098 seconds
Running: read_and_compute_torchaudio ...
Audio: tensor([[ -5103,  23502],
        [  4449, -11882],
        [ 26703,  20063],
        ...,
        [  7883, -29269],
        [ 12201, -30454],
        [ 30524,  31554]], dtype=torch.int16)
Other: 37
Total Execution Time: 0.41034483909606934 seconds


In [7]:
audio_generator = AudioGenerator(sample_rate=44100, duration=60 * 30, channels=2)

%timeit audio_data = fastwave.read(audio_generator.file_path, mode=fastwave.ReadMode.DEFAULT)

%timeit audio_data = wavfile.read(audio_generator.file_path)

%timeit sig, _ = torchaudio.load(audio_generator.file_path, normalize=False, channels_first=False)

audio_generator.delete_generated_file()

61.1 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
48 ms ± 960 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
318 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
